In [49]:
import time
import time, psutil, os
start = time.time()
m0 = psutil.Process(os.getpid()).memory_info().rss / 1024**2

In [50]:
import re
import unidecode
import polars as pl
import pandas as pd
import numpy as np
from rapidfuzz import fuzz


In [51]:
import polars as pl

ficheiro = r"C:\Users\1420844\Downloads\addressGeoLoc_export_20221006.csv"

df = pl.read_csv(
    ficheiro,
    separator=",",
    quote_char='"',
    skip_rows=1,
    has_header=False,
    new_columns=["MORADA", "CP", "LOCALIDADE", "LATITUDE", "LONGITUDE"],
    decimal_comma=True
)

pl.Config.set_tbl_formatting("UTF8_FULL")  # formato mais limpo
pl.Config.set_tbl_rows(5)                 # nº de linhas a mostrar
pl.Config.set_tbl_width_chars(120) 
pl.Config.set_tbl_cols("10_000")    



polars.config.Config

In [52]:
df.head()

MORADA,CP,LOCALIDADE,LATITUDE,LONGITUDE
str,str,str,str,str
"""RUA CESARIO VERDE LOTE 3 A DAS…","""2660""","""FRIELAS""","""38.852697456""","""-9.16971532999997"""
"""RUA DO ROXICO NR50""","""3865-110""","""FERMELA""","""40.709808191""","""-8.54878568099997"""
"""AVENIDA 13 DE MAIO N 536""","""3885-227""","""CORTEGACA OVR""","""40.94109138""","""-8.61864986999996"""
"""AV REINALDO SANTOS N 24 3 DTO""","""2675-673""","""ODIVELAS""","""38.7926955990001""","""-9.18881750599996"""
"""RUA DO BARREIRO N 547""","""4405-730""","""VILA NOVA DE GAIA""","""41.113905909""","""-8.63062512299996"""


In [53]:
# garantir que CP é string
df = df.with_columns(df["CP"].cast(pl.Utf8))

# filtrar e contar
contagem = (
    df.filter(df["CP"].str.slice(0,4).is_in(["4830", "4850"]))
      .select(pl.count("MORADA"))
      .item()
)

print("Número de moradas:", contagem)

Número de moradas: 1158


In [54]:
df_FV = pl.DataFrame({ "MORADA": ["R. Cmdt Luis Pinto 37", "Av. da Capela 147"], "CP": ["4830", "4830"], "LOCALIDADE": ["Póvoa de Lanhoso", "Póvoa de Lanhoso"], "LATITUDE": ["41.5739", "41.6029"], "LONGITUDE": ["-8.2684", "-8.3133"] })

In [55]:
df_filtrado = df.filter(
    df["CP"].str.slice(0, 4).is_in(["4830", "4850"])
)

df_filtrado

MORADA,CP,LOCALIDADE,LATITUDE,LONGITUDE
str,str,str,str,str
"""RUA COMANDANTE LUIS PINTO DA S…","""4830""","""POVOA DE LANHOSO""","""41.573881""","""-8.268431"""
"""AVENIDA DA CAPELA 147""","""4830""","""POVOA DE LANHOSO""","""41.602910728""","""-8.31333931399996"""
"""RUA NEUVES MAISONS NUMERO 211 …","""4830-550""","""POVOA DE LANHOSO""","""41.573046436""","""-8.26624732199997"""
…,…,…,…,…
"""RUA DA QUINTA N 179 B""","""4830-379""","""LOUREDO PVL""","""41.5465713200001""","""-8.28988375599994"""
"""RUA 5 DE SETEMBRO N94 N94 RC E…","""4830""","""PA VOA DE LANHOSO""","""41.57924""","""-8.273131"""


In [56]:
import numpy as np

def haversine(lat1, lon1, lat2, lon2):
    # converter para radianos
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    # fórmula haversine
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2
    c = 2 * np.arcsin(np.sqrt(a))
    R = 6371000  # raio da Terra em metros
    return R * c


In [57]:
# converter para pandas para facilitar merge
pdf1 = df_filtrado.to_pandas().copy()
pdf2 = df_FV.to_pandas().copy()

# converter coordenadas para float
pdf1["LATITUDE"] = pdf1["LATITUDE"].astype(float)
pdf1["LONGITUDE"] = pdf1["LONGITUDE"].astype(float)
pdf2["LATITUDE"] = pdf2["LATITUDE"].astype(float)
pdf2["LONGITUDE"] = pdf2["LONGITUDE"].astype(float)

# criar todas as combinações (produto cartesiano)
pdf1["key"] = 1
pdf2["key"] = 1
pairs = pdf1.merge(pdf2, on="key", suffixes=("_df1", "_df2")).drop("key", axis=1)

# calcular distância
pairs["dist_m"] = haversine(
    pairs["LATITUDE_df1"], pairs["LONGITUDE_df1"],
    pairs["LATITUDE_df2"], pairs["LONGITUDE_df2"]
)

# filtrar ≤ 500 metros
pairs_500 = pairs[pairs["dist_m"] <= 500].reset_index(drop=True)
pairs_500


,MORADA_df1,CP_df1,LOCALIDADE_df1,LATITUDE_df1,LONGITUDE_df1,MORADA_df2,CP_df2,LOCALIDADE_df2,LATITUDE_df2,LONGITUDE_df2,dist_m
0,RUA COMANDANTE LUIS PINTO DA SILVA NR 37,4830,POVOA DE LANHOSO,41.573881,-8.268431,R. Cmdt Luis Pinto 37,4830,Póvoa de Lanhoso,41.5739,-8.2684,3.333675
1,AVENIDA DA CAPELA 147,4830,POVOA DE LANHOSO,41.602911,-8.313339,Av. da Capela 147,4830,Póvoa de Lanhoso,41.6029,-8.3133,3.479725
2,RUA NEUVES MAISONS NUMERO 211 1 DIREITO,4830-550,POVOA DE LANHOSO,41.573046,-8.266247,R. Cmdt Luis Pinto 37,4830,Póvoa de Lanhoso,41.5739,-8.2684,202.669535
3,RUA COMANDANTE LUIS PINTO DA SILVA N 123 2 ESQ...,4830,POVOA DE LANHOSO,41.573520,-8.268890,R. Cmdt Luis Pinto 37,4830,Póvoa de Lanhoso,41.5739,-8.2684,58.709846
4,AVD 25 DE ABRIL 60,4830,PA VOA DE LANHOSO,41.575171,-8.270491,R. Cmdt Luis Pinto 37,4830,Póvoa de Lanhoso,41.5739,-8.2684,224.116641
...,...,...,...,...,...,...,...,...,...,...,...
218,RUA 25 DE NOVEMBRO 84 CASA,4830,POVOA DE LANHOSO,41.573208,-8.271709,R. Cmdt Luis Pinto 37,4830,Póvoa de Lanhoso,41.5739,-8.2684,285.796956
219,RUA DR AVELINO PEREIRA DE CARVALHO 81 3 DRT BRA,4830,POVOA DE LANHOSO,41.575619,-8.271935,R. Cmdt Luis Pinto 37,4830,Póvoa de Lanhoso,41.5739,-8.2684,350.719326
220,R DR AVELINO PEREIRA DE CARVALHO 67,4830,POVOA DE LANHOSO,41.575703,-8.271837,R. Cmdt Luis Pinto 37,4830,Póvoa de Lanhoso,41.5739,-8.2684,349.156506
221,RUA NEUVES MAISONS 95 RC,4830-550,POVOA DE LANHOSO,41.573347,-8.267586,R. Cmdt Luis Pinto 37,4830,Póvoa de Lanhoso,41.5739,-8.2684,91.459575


In [58]:

# ==========================
# 1. Dicionários
# ==========================
abreviaturas = {
    "cmdt": "comandante", "cmte": "comandante",
    "dr": "doutor", "dra": "doutora",
    "sr": "senhor", "sra": "senhora", "srª": "senhora",
    "eng": "engenheiro", "enga": "engenheira", "engª": "engenheira",
    "prof": "professor", "profa": "professora", "profª": "professora",
    "arq": "arquiteto", "arqa": "arquiteta", "arqª": "arquiteta",
    "cap": "capitao", "maj": "major", "gen": "general",
    "ten": "tenente", "alm": "almirante",
    "sta": "santa", "sto": "santo", "s": "sao", "sao": "sao",
    "ns": "nossa senhora", "n s": "nossa senhora",
    "nsr": "nosso senhor", "n sr": "nosso senhor",
    "dom": "dom", "d": "dom",
    "visc": "visconde", "cond": "conde", "marq": "marques", "bar": "barao"
}

mapa_logradouros = {
    "r": "rua", "rua": "rua", "rª": "rua", "ruela": "ruela",
    "av": "avenida", "avd": "avenida", "avda": "avenida", "avenida": "avenida",
    "tv": "travessa", "trs": "travessa", "trav": "travessa", "travessa": "travessa",
    "pc": "praca", "pç": "praca", "pr": "praca", "prç": "praca", "praca": "praca",
    "lg": "largo", "largo": "largo",
    "al": "alameda", "alameda": "alameda",
    "bq": "beco", "beco": "beco",
    "esc": "escadas", "escadinha": "escadinha", "escadaria": "escadaria",
    "estr": "estrada", "estrada": "estrada",
    "cm": "caminho", "caminho": "caminho", "cam": "caminho",
    "cç": "calcada", "calcada": "calcada", "calçada": "calcada",
    "qt": "quinta", "quinta": "quinta",
    "br": "bairro", "bairro": "bairro",
    "rot": "rotunda", "rotunda": "rotunda",
    "via": "via", "passeio": "passeio",
    "urb": "urbanizacao", "urbanizacao": "urbanizacao",
    "zona": "zona", "lgd": "lugar", "lugar": "lugar",
    "campo": "campo", "terreiro": "terreiro"
}

# ==========================
# 2. Funções de normalização
# ==========================
def normalizar(texto: str) -> str:
    if not texto:
        return ""
    txt = unidecode.unidecode(str(texto).lower())
    txt = txt.replace(".", "").strip()
    palavras = txt.split()
    palavras = [abreviaturas.get(p, mapa_logradouros.get(p, p)) for p in palavras]
    return " ".join(palavras)

def limpar_nome_rua(nome: str) -> str:
    if not nome:
        return ""
    txt = unidecode.unidecode(str(nome).lower())
    # remover logradouros
    for log in set(mapa_logradouros.values()):
        txt = re.sub(rf"\b{log}\b", "", txt)
    # remover tokens de número
    txt = re.sub(r"\b(n|nr|nº|nro|numero|num)\b", "", txt)
    txt = re.sub(r"\d+", "", txt)
    # expandir abreviaturas
    palavras = txt.split()
    palavras = [abreviaturas.get(p, p) for p in palavras]
    txt = " ".join(palavras)
    txt = re.sub(r"\s+", " ", txt).strip()
    return txt

# ==========================
# 3. Extrair + normalizar partes
# ==========================
def extrair_partes(df: pl.DataFrame) -> pl.DataFrame:
    # Extrair componentes com regex
    df_parts = df.with_columns([
        # logradouro (case-insensitive, aceita abreviações R., Av., Tv.)
        pl.col("MORADA").str.extract(
            r"(?i)^(R\.?|RUA|AV\.?|AVENIDA|TRAV\.?|TRAVESSA|PRAÇA|ESTRADA|BECO|ALAMEDA|LARGO|CALÇADA|ROTUNDA)", 1
        ).alias("logradouro"),

        # nome da rua
        pl.col("MORADA").str.extract(
            r"(?i)^(?:R\.?|RUA|AV\.?|AVENIDA|TRAV\.?|TRAVESSA|PRAÇA|ESTRADA|BECO|ALAMEDA|LARGO|CALÇADA|ROTUNDA)\s+([A-ZÇÃÕÉÓÀ\s]+?)(?:\d|$)", 
            1
        ).str.strip_chars().alias("nome_rua"),

        # número
        pl.col("MORADA").str.extract(r"\s(\d+[A-Z]?)\b", 1).alias("numero"),

        # andar
        pl.col("MORADA").str.extract(r"(\d+º\s*[A-Z]*|R/C)", 1).alias("andar"),

        # código postal
        pl.col("MORADA").str.extract(r"(\d{4}-\d{3})", 1).alias("codigo_postal"),

        # localidade (usar coluna LOCALIDADE quando existir)
        pl.when(pl.col("LOCALIDADE").is_not_null())
          .then(pl.col("LOCALIDADE"))
          .otherwise(pl.col("MORADA").str.extract(r"([A-ZÇÃÕÉÓ]+)$", 1))
          .alias("localidade")
    ])

    # Aplicar normalização
    df_parts = df_parts.with_columns([
        pl.col("logradouro").map_elements(normalizar, return_dtype=pl.Utf8).alias("logradouro_norm"),
        pl.col("nome_rua").map_elements(limpar_nome_rua, return_dtype=pl.Utf8).alias("nome_rua_norm"),
        pl.col("numero").map_elements(normalizar, return_dtype=pl.Utf8).alias("numero_norm"),
        pl.col("andar").map_elements(normalizar, return_dtype=pl.Utf8).alias("andar_norm"),
        pl.col("codigo_postal").map_elements(normalizar, return_dtype=pl.Utf8).alias("codigo_postal_norm"),
        pl.col("localidade").map_elements(normalizar, return_dtype=pl.Utf8).alias("localidade_norm")
    ])
    
    return df_parts



In [59]:
# aplicar extrair_partes às moradas de cada lado
df1_parts = extrair_partes(pl.DataFrame({
    "MORADA": pairs_500["MORADA_df1"],
    "LOCALIDADE": pairs_500["LOCALIDADE_df1"]
}))

df2_parts = extrair_partes(pl.DataFrame({
    "MORADA": pairs_500["MORADA_df2"],
    "LOCALIDADE": pairs_500["LOCALIDADE_df2"]
}))


In [60]:
def comparar_dataframes(df1: pl.DataFrame, df2: pl.DataFrame, threshold=85):
    results = []
    pdf1, pdf2 = df1.to_pandas(), df2.to_pandas()  # converte para pandas

    for i, row1 in pdf1.iterrows():
        for j, row2 in pdf2.iterrows():
            score_log = fuzz.token_sort_ratio(row1["logradouro_norm"], row2["logradouro_norm"])
            score_rua = fuzz.token_sort_ratio(row1["nome_rua_norm"], row2["nome_rua_norm"])
            score_num = fuzz.token_sort_ratio(str(row1["numero_norm"]), str(row2["numero_norm"]))
            score_loc = fuzz.token_sort_ratio(row1["localidade_norm"], row2["localidade_norm"])

            score_final = (0.3*score_log + 0.4*score_rua + 0.2*score_num + 0.1*score_loc)

            if score_final >= threshold:
                results.append({
                    "id_df1": i, "id_df2": j,
                    "score": round(score_final, 2),
                    "logradouro1": row1["logradouro_norm"], "logradouro2": row2["logradouro_norm"],
                    "nome_rua1": row1["nome_rua_norm"], "nome_rua2": row2["nome_rua_norm"],
                    "numero1": row1["numero_norm"], "numero2": row2["numero_norm"],
                    "localidade1": row1["localidade_norm"], "localidade2": row2["localidade_norm"]
                })
    return pd.DataFrame(results)


In [61]:
df_matches = comparar_dataframes(df1_parts, df2_parts, threshold=85)

# juntar de volta a distância em metros
df_matches = df_matches.merge(
    pairs_500.reset_index().rename(columns={"index":"id_pair"})[["id_pair","dist_m"]],
    left_on=["id_df1","id_df2"], right_on=["id_pair","id_pair"], how="left"
).drop("id_pair", axis=1)

df_matches


,id_df1,id_df2,score,logradouro1,logradouro2,nome_rua1,nome_rua2,numero1,numero2,localidade1,localidade2,dist_m
0,0,0,92.94,rua,rua,comandante luis pinto da silva,comandante luis pinto,37,37,povoa de lanhoso,povoa de lanhoso,3.333675
1,0,2,92.94,rua,rua,comandante luis pinto da silva,comandante luis pinto,37,37,povoa de lanhoso,povoa de lanhoso,NaN
2,0,3,92.94,rua,rua,comandante luis pinto da silva,comandante luis pinto,37,37,povoa de lanhoso,povoa de lanhoso,NaN
3,0,4,92.94,rua,rua,comandante luis pinto da silva,comandante luis pinto,37,37,povoa de lanhoso,povoa de lanhoso,NaN
4,0,5,92.94,rua,rua,comandante luis pinto da silva,comandante luis pinto,37,37,povoa de lanhoso,povoa de lanhoso,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
662,164,218,85.00,rua,rua,comandante luis pinto silva,comandante luis pinto,70,37,povoa de lanhoso,povoa de lanhoso,NaN
663,164,219,85.00,rua,rua,comandante luis pinto silva,comandante luis pinto,70,37,povoa de lanhoso,povoa de lanhoso,NaN
664,164,220,85.00,rua,rua,comandante luis pinto silva,comandante luis pinto,70,37,povoa de lanhoso,povoa de lanhoso,NaN
665,164,221,85.00,rua,rua,comandante luis pinto silva,comandante luis pinto,70,37,povoa de lanhoso,povoa de lanhoso,NaN


In [63]:
df_matches[df_matches["dist_m"].notna()]


,id_df1,id_df2,score,logradouro1,logradouro2,nome_rua1,nome_rua2,numero1,numero2,localidade1,localidade2,dist_m
0,0,0,92.94,rua,rua,comandante luis pinto da silva,comandante luis pinto,37,37,povoa de lanhoso,povoa de lanhoso,3.333675
222,1,1,100.00,avenida,avenida,da capela,da capela,147,147,povoa de lanhoso,povoa de lanhoso,3.479725
299,77,77,88.00,rua,rua,comandante luis pinto,comandante luis pinto,123,37,povoa de lanhoso,povoa de lanhoso,58.619899
608,164,164,85.00,rua,rua,comandante luis pinto silva,comandante luis pinto,70,37,povoa de lanhoso,povoa de lanhoso,37.977748


In [64]:
df_matches[df_matches["dist_m"].isna()]


,id_df1,id_df2,score,logradouro1,logradouro2,nome_rua1,nome_rua2,numero1,numero2,localidade1,localidade2,dist_m
1,0,2,92.94,rua,rua,comandante luis pinto da silva,comandante luis pinto,37,37,povoa de lanhoso,povoa de lanhoso,NaN
2,0,3,92.94,rua,rua,comandante luis pinto da silva,comandante luis pinto,37,37,povoa de lanhoso,povoa de lanhoso,NaN
3,0,4,92.94,rua,rua,comandante luis pinto da silva,comandante luis pinto,37,37,povoa de lanhoso,povoa de lanhoso,NaN
4,0,5,92.94,rua,rua,comandante luis pinto da silva,comandante luis pinto,37,37,povoa de lanhoso,povoa de lanhoso,NaN
5,0,6,92.94,rua,rua,comandante luis pinto da silva,comandante luis pinto,37,37,povoa de lanhoso,povoa de lanhoso,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
662,164,218,85.00,rua,rua,comandante luis pinto silva,comandante luis pinto,70,37,povoa de lanhoso,povoa de lanhoso,NaN
663,164,219,85.00,rua,rua,comandante luis pinto silva,comandante luis pinto,70,37,povoa de lanhoso,povoa de lanhoso,NaN
664,164,220,85.00,rua,rua,comandante luis pinto silva,comandante luis pinto,70,37,povoa de lanhoso,povoa de lanhoso,NaN
665,164,221,85.00,rua,rua,comandante luis pinto silva,comandante luis pinto,70,37,povoa de lanhoso,povoa de lanhoso,NaN


In [65]:
df_matches_valid = df_matches[df_matches["dist_m"].notna()].reset_index(drop=True)
df_matches_valid


,id_df1,id_df2,score,logradouro1,logradouro2,nome_rua1,nome_rua2,numero1,numero2,localidade1,localidade2,dist_m
0,0,0,92.94,rua,rua,comandante luis pinto da silva,comandante luis pinto,37,37,povoa de lanhoso,povoa de lanhoso,3.333675
1,1,1,100.00,avenida,avenida,da capela,da capela,147,147,povoa de lanhoso,povoa de lanhoso,3.479725
2,77,77,88.00,rua,rua,comandante luis pinto,comandante luis pinto,123,37,povoa de lanhoso,povoa de lanhoso,58.619899
3,164,164,85.00,rua,rua,comandante luis pinto silva,comandante luis pinto,70,37,povoa de lanhoso,povoa de lanhoso,37.977748


In [62]:
m1 = psutil.Process(os.getpid()).memory_info().rss / 1024**2
print("Tempo Polars:", round(time.time() - start, 2), "segundos")
print("Memória Polars:", round(m1 - m0, 2), "MB")

Tempo Polars: 4.6 segundos
Memória Polars: 132.93 MB
